In [2]:
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("torch.version.cuda:", torch.version.cuda)

if torch.cuda.is_available():
    print("device:", torch.cuda.get_device_name(0))


torch: 2.7.1+cu118
cuda available: True
torch.version.cuda: 11.8
device: Quadro RTX 4000


/mnt/storage/miniconda3/envs/kpconv/lib/python3.10/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
# CELL 1 — CHECK ENVIRONMENT

import sys, platform, os

print("Python executable :", sys.executable)
print("Python version    :", sys.version.splitlines()[0])
print("Platform          :", platform.platform())
print("Working directory :", os.getcwd())

def test_import(name):
    try:
        module = __import__(name)
        version = getattr(module, "__version__", "OK (no version attribute)")
        print(f"{name:15} : OK ({version})")
    except Exception as e:
        print(f"{name:15} : ERROR -> {e}")

print("\n--- Checking installed packages ---")
test_import("torch")
test_import("numpy")
test_import("scipy")
test_import("sklearn")
test_import("tqdm")
test_import("plyfile")
test_import("open3d")


Python executable : /mnt/storage/miniconda3/envs/kpconv/bin/python
Python version    : 3.10.19 (main, Oct 21 2025, 16:43:05) [GCC 11.2.0]
Platform          : Linux-6.8.0-85-generic-x86_64-with-glibc2.35
Working directory : /mnt/storage/SSS_03/CAPSTONE/CAPSTONE_SSS_03

--- Checking installed packages ---
torch           : OK (2.7.1+cu118)
numpy           : OK (2.2.6)
scipy           : OK (1.15.3)
sklearn         : OK (1.7.2)
tqdm            : OK (4.66.1)
plyfile         : ERROR -> No module named 'plyfile'
open3d          : ERROR -> No module named 'open3d'


In [4]:
# CELL 2 — INSTALL MISSING PACKAGES (plyfile + open3d)

import sys, subprocess

print("Installing into interpreter:", sys.executable)

def pip_install(pkg):
    print(f"\n>>> Installing: {pkg}")
    out = subprocess.run(
        [sys.executable, "-m", "pip", "install", pkg],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    print(out.stdout)
    if out.returncode != 0:
        print("ERROR:", out.stderr)

# install missing packages
pip_install("plyfile")
pip_install("open3d")

print("\n>>> Installation complete.")
print(">>> NOW restart the kernel (VS Code: Command Palette → 'Notebook: Restart Kernel').")


Installing into interpreter: /mnt/storage/miniconda3/envs/kpconv/bin/python

>>> Installing: plyfile
  Using cached plyfile-1.1.3-py3-none-any.whl.metadata (43 kB)
Using cached plyfile-1.1.3-py3-none-any.whl (36 kB)


>>> Installing: open3d
  Using cached open3d-0.19.0-cp310-cp310-manylinux_2_31_x86_64.whl.metadata (4.3 kB)
  Using cached dash-3.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached configargparse-1.7.1-py3-none-any.whl.metadata (24 kB)
  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached pyquaternion-0.9.9-py3-none-any.whl.metadata (1.4 kB)
  Using cached plotly-6.4.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached retrying-1.4.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached widgetsnbextension-4.0.15-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.16-py3-none-any.whl.metadata

In [3]:
# CELL 3 — VERIFY PACKAGES (run AFTER kernel restart)

import torch, numpy as np, sklearn
import open3d as o3d
from plyfile import PlyData

print("torch version      :", torch.__version__)
print("cuda available     :", torch.cuda.is_available())
print("cuda version       :", torch.version.cuda if torch.cuda.is_available() else None)
print("numpy version      :", np.__version__)
print("sklearn version    :", sklearn.__version__)
print("open3d version     :", o3d.__version__)
print("plyfile            : OK")

# Set DEVICE for later training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\nDEVICE =", DEVICE)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
torch version      : 2.7.1+cu118
cuda available     : True
cuda version       : 11.8
numpy version      : 2.2.6
sklearn version    : 1.7.2
open3d version     : 0.19.0
plyfile            : OK

DEVICE = cuda


In [7]:
# CELL 4 — PLY Dataset Loader (for your 3-class dataset)

import numpy as np
import torch
from torch.utils.data import Dataset
from pathlib import Path
from plyfile import PlyData

DATA_DIR = Path("/mnt/storage/SSS_03/DATA/train_sphere_ascii_roi")

label_map = {1: 0, 3: 1, 9: 2}   # final correct mapping

def read_ply_hungary(path):
    pd = PlyData.read(str(path))
    v = pd["vertex"].data

    pts = np.vstack([v["x"], v["y"], v["z"]]).T.astype(np.float32)
    labels = np.array(v["scalar_NewClassification"]).astype(np.int64)

    # remap labels
    labels = np.vectorize(lambda x: label_map.get(int(x), 255))(labels)
    mask = labels != 255
    return pts[mask], labels[mask]

class HungaryPLYDataset(Dataset):
    def __init__(self, root, files, points_per_sample=2048, augment=True):
        self.root = Path(root)
        self.files = files
        self.points_per_sample = points_per_sample
        self.augment = augment

    def __len__(self):
        return len(self.files) * 10

    def __getitem__(self, idx):
        ply = self.files[np.random.randint(len(self.files))]
        pts, labels = read_ply_hungary(self.root / ply)

        N = len(pts)
        P = self.points_per_sample
        idxs = np.random.choice(N, P, replace=(N < P))

        pts = pts[idxs]
        labels = labels[idxs]

        if self.augment:
            theta = np.random.uniform(0, 2 * np.pi)
            R = np.array([
                [np.cos(theta), -np.sin(theta), 0],
                [np.sin(theta),  np.cos(theta), 0],
                [0, 0, 1]
            ], dtype=np.float32)
            pts = pts @ R.T
            pts += np.random.normal(0, 0.01, pts.shape)

        pts -= pts.mean(0, keepdims=True)

        return torch.from_numpy(pts), torch.from_numpy(labels)

def collate_batch(batch):
    pts = torch.stack([b[0] for b in batch], dim=0)
    lbl = torch.stack([b[1] for b in batch], dim=0)
    return pts, lbl

# Create train/val split
all_files = sorted([f.name for f in DATA_DIR.glob("*.ply")])
import random
random.shuffle(all_files)
N = len(all_files)
train_files = all_files[:int(0.8*N)]
val_files = all_files[int(0.8*N):]




train_ds = HungaryPLYDataset(DATA_DIR, train_files, points_per_sample=1024, augment=True)
val_ds   = HungaryPLYDataset(DATA_DIR, val_files,   points_per_sample=1024, augment=False)

from torch.utils.data import DataLoader
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True,
                          num_workers=0, collate_fn=collate_batch, pin_memory=True)

val_loader   = DataLoader(val_ds, batch_size=1, shuffle=False,
                          num_workers=0, collate_fn=collate_batch, pin_memory=True)
print("train batches:", len(train_loader), "val batches:", len(val_loader))


train batches: 120 val batches: 40


In [8]:
# CELL 5 — FIXED KPConv-Like Model (Dimension-Safe)

import torch
import torch.nn as nn
import torch.nn.functional as F

def knn(pts, K=8):
    """
    pts: (B, P, 3)
    returns idx: (B, P, K)
    """
    dist = torch.cdist(pts, pts)  # (B, P, P)
    idx = dist.topk(K, largest=False)[1]  # (B, P, K)
    return idx

class KPConvLayer(nn.Module):
    def __init__(self, in_c, out_c, K=8):
        super().__init__()
        self.K = K
        self.mlp = nn.Sequential(
            nn.Linear(in_c + 3, out_c),
            nn.ReLU(),
            nn.Linear(out_c, out_c)
        )

    def forward(self, pts, feats):
        """
        pts:   (B, P, 3)
        feats: (B, P, C)
        output: (B, P, out_c)
        """
        B, P, C = feats.shape
        K = self.K

        idx = knn(pts, K)         # (B, P, K)

        # gather neighbor coords: (B, P, K, 3)
        pts_expand = pts.unsqueeze(2).expand(B, P, K, 3)
        neigh_pts = torch.gather(pts.unsqueeze(1).expand(B, P, P, 3), 
                                 2, 
                                 idx.unsqueeze(-1).expand(B, P, K, 3))

        # gather neighbor feats: (B, P, K, C)
        feats_expand = feats.unsqueeze(1).expand(B, P, P, C)
        neigh_feats = torch.gather(feats_expand,
                                   2,
                                   idx.unsqueeze(-1).expand(B, P, K, C))

        # compute relative positions
        rel = neigh_pts - pts_expand  # (B,P,K,3)

        # concatenate features: (B,P,K, 3+C)
        inp = torch.cat([rel, neigh_feats], dim=-1)

        # MLP → (B,P,K,out_c)
        out = self.mlp(inp)

        # Max over K neighbors → (B,P,out_c)
        out = out.max(dim=2)[0]

        return out


class KPNet(nn.Module):
    def __init__(self, base=16, num_classes=3, K=8):
        super().__init__()
        self.fc0 = nn.Linear(3, base)

        self.kp1 = KPConvLayer(base, base*2, K)
        self.kp2 = KPConvLayer(base*2, base*4, K)

        self.head = nn.Sequential(
            nn.Linear(base*4, base*4),
            nn.ReLU(),
            nn.Linear(base*4, num_classes)
        )

    def forward(self, pts):
        feats = F.relu(self.fc0(pts))
        feats = self.kp1(pts, feats)
        feats = self.kp2(pts, feats)
        out = self.head(feats)
        return out

model = KPNet(base=8, num_classes=3, K=6).to(DEVICE)
model


KPNet(
  (fc0): Linear(in_features=3, out_features=8, bias=True)
  (kp1): KPConvLayer(
    (mlp): Sequential(
      (0): Linear(in_features=11, out_features=16, bias=True)
      (1): ReLU()
      (2): Linear(in_features=16, out_features=16, bias=True)
    )
  )
  (kp2): KPConvLayer(
    (mlp): Sequential(
      (0): Linear(in_features=19, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
    )
  )
  (head): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [9]:
# Sanity Test — Forward + Backward on 1 batch
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

pts, lbl = next(iter(train_loader))
pts, lbl = pts.to(DEVICE), lbl.to(DEVICE)

optimizer.zero_grad()
out = model(pts)
loss = criterion(out.view(-1,3), lbl.view(-1))
loss.backward()
optimizer.step()

print("Sanity OK — loss =", loss.item())


Sanity OK — loss = 1.1420948505401611


In [5]:
# CELL 6 — BEST TRAINING LOOP WITH TQDM
from tqdm import tqdm

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

def compute_iou(pred, gt, nc):
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(gt, pred, labels=list(range(nc)))
    ious = []
    for i in range(nc):
        tp = cm[i,i]
        fp = cm[:,i].sum() - tp
        fn = cm[i,:].sum() - tp
        denom = tp + fp + fn
        ious.append(tp / denom if denom > 0 else 0)
    return np.array(ious)

EPOCHS = 10
best_miou = 0

for ep in range(EPOCHS):
    model.train()
    loop = tqdm(train_loader, desc=f"Epoch {ep+1}/{EPOCHS}", leave=True)

    for pts, lbl in loop:
        pts, lbl = pts.to(DEVICE), lbl.to(DEVICE)

        optimizer.zero_grad()
        logits = model(pts)
        loss = criterion(logits.view(-1,3), lbl.view(-1))
        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())

    # Validation
    model.eval()
    preds_all, lbl_all = [], []
    with torch.no_grad():
        for pts, lbl in val_loader:
            pts = pts.to(DEVICE)
            logits = model(pts)
            pred = logits.argmax(-1).cpu().numpy().reshape(-1)
            lbl = lbl.numpy().reshape(-1)
            preds_all.append(pred)
            lbl_all.append(lbl)

    preds_all = np.concatenate(preds_all)
    lbl_all = np.concatenate(lbl_all)
    iou = compute_iou(preds_all, lbl_all, 3)
    miou = iou.mean()

    print(f"\nEpoch {ep+1} → mIoU={miou:.4f} | IoUs={iou}")

    if miou > best_miou:
        best_miou = miou
        torch.save(model.state_dict(), "best_kpconv.pth")
        print("Saved BEST model!")

print("\nTRAINING DONE.")
print("Best mIoU achieved:", best_miou)


Epoch 1/10:  98%|█████████▊| 117/120 [42:12<01:04, 21.64s/it, loss=0.258] 


KeyboardInterrupt: 